In [ ]:
import os
import datetime
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.tsa.stattools as ts

In [ ]:
read_csv = False
s1218 = 1
years = 2015 - 1990 + 0.5

In [ ]:
# Ordenando os melhores pares
def top_coint_pairs(data,pvalue_matrix,gamma, alpha, n): 
#alpha = nivel de significancia para o teste ADF
#n = top n ativos com o menor pvalue    
    alpha_filter = np.where(pvalues < alpha)
    pvalues_f = pvalues[alpha_filter] # pvalores menores que alpha
    #print(f"Alpha filter rows len {len(alpha_filter[0])} | cols len {len(alpha_filter[1])} | value {alpha_filter}")
    #print(f"pvalues_f len: {np.shape(pvalues_f)} | value: {pvalues_f}")
    stock_a = data.columns[list(alpha_filter)[0]] # relacionando o pvalor com a ação A
    stock_b = data.columns[list(alpha_filter)[1]] # relacionando o pvalor com a ação B
    gammas_f = gammas[alpha_filter] # relacionando o pvalor com o gamma
    N = len(list(alpha_filter[0])) # quantidade de pares cointegrados

    d = []
    for i in range(N):
        pair_dict = {
            'Stock A': stock_a[i],
            'Stock B': stock_b[i],
            'P-Values': pvalues_f[i],
            'Gamma': gammas_f[i]
        }
        #if(i%2000 == 0):
        #print(f"Appending pair dict: {pair_dict}")
        d.append(pair_dict)

    return pd.DataFrame(d).sort_values(by="P-Values").iloc[:n,]


# Calcula os retornos da carteira e armazenando em um data frame
def calculate_profit(pair, spread, threshold, par1, par2, resumo, semester, gamma):
    #print(f"Calculating profits for pair {par1}-{par2}")
    
    #print(f"Spread series: {spread}")
    
    date_format = "%Y-%m-%d"
    
    log_ret = spread.diff() # log return eh o incremento
    #print(f"log_ret len: {len(log_ret)} | variable: {log_ret}")
    dias = spread.index
    #print(f"Dias len: {len(dias)} | variable: {dias}")
    z_score = (spread-spread.mean())/spread.std()
    z_score.plot()
    #print(f"Z-score index: {z_score.index} | variable: {z_score}")
    portfolio_return = []
    pos = 0 # 0: sem posição aberta
            # 1: Comprei o meu portfolio h = (1,-gamma)
            # -1: Vendi o meu portfolio h = -(1,-gamma)

    dias_abertura = []
    dias_fechamento = []

    count = 0
    dia_abertura = 0
    dia_fechamento = 0
    
    closing_threshold = 0.75
    
    for i in z_score.index:   
        if (z_score[i] > threshold) and (pos == 0):
            # Posição fechada no par e com sinal short 1st e long 2nd
            pos = -1

            count += 1
            dia_abertura = dias[i - dias[0]]
            retornos_op = []


        elif (z_score[i] < -threshold)  and (pos == 0):
            # Posição fechada no par e com sinal de long 1st e short 2nd 
            pos = 1            

            count += 1
            dia_abertura = dias[i - dias[0]]
            retornos_op = []

        else:
            #print(f"Dia {i} | Pos {pos} | log_ret {log_ret[i]} | S1 return {returns[par1][i]} | S2 return {returns[par2][i]} | Net {pos*(returns[par1][i] - gamma*returns[par2][i])}")
            if (pos == 1) and (z_score[i] < -closing_threshold):
                # Posição vendidada aberta no par aberta e sem convergência
                portfolio_return.append(log_ret[i]*pos)
                retornos_op.append(log_ret[i]*pos)
                
                Rpair[i-1, pair] = log_ret[i]*pos


            elif (pos == 1) and (z_score[i] >= -closing_threshold):
                # Posição vendida aberta no par e com sinal de convergência
                portfolio_return.append(log_ret[i]*pos)
                pos = 0

                dia_fechamento = dias[i - dias[0]]
                delta_dias = dia_fechamento - dia_abertura
                retornos_op.append(log_ret[i]*pos)
                retorno_op = pd.Series(retornos_op).sum()

                Rpair[i-1, pair] = log_ret[i]*pos
                
                resumo.append([count, semester, dia_abertura, dia_fechamento, delta_dias, retorno_op, par1, par2, True])

            
            elif (pos == -1) and (z_score[i] > closing_threshold):
                portfolio_return.append(log_ret[i]*pos)
                retornos_op.append(log_ret[i]*pos)

                Rpair[i-1, pair] = log_ret[i]*pos
                

            elif (pos == -1) and (z_score[i] <= closing_threshold):
                portfolio_return.append(log_ret[i]*pos)
                pos = 0

                dia_fechamento = dias[i - dias[0]]
                delta_dias = dia_fechamento - dia_abertura
                retornos_op.append(log_ret[i]*pos)
                retorno_op = pd.Series(retornos_op).sum()

                Rpair[i-1, pair] = log_ret[i]*pos
                
                resumo.append([count, semester, dia_abertura, dia_fechamento, delta_dias, retorno_op, par1, par2, True])

            else:
                # Sem posição aberta e nem sinal de entrada
                
                if pos != 0:
                    dia_fechamento = dias[i - dias[0]]
                    delta_dias = dia_fechamento - dia_abertura
                    retornos_op.append(log_ret[i]*pos)
                    retorno_op = pd.Series(retornos_op).sum()

                    Rpair[i-1, pair] = log_ret[i]*pos
                    
                    resumo.append([count, semester, dia_abertura, dia_fechamento, delta_dias, retorno_op, par1, par2, True])

                pos = 0

    if pos != 0:
        # Operação sem convergência
        pos = 0

        dia_fechamento = i
        delta_dias = dia_fechamento - dia_abertura
        retorno_op = pd.Series(retornos_op).sum()

        resumo.append([count, semester, dia_abertura, dia_fechamento, delta_dias, retorno_op, par1, par2, False])
                
    #print(f"Total return: {sum(pair_returns)} | Pair returns: {pair_returns}")
    #print(f"Conversão do par: {pos}")    
    total_ret = pd.Series(portfolio_return).sum()

    return total_ret, resumo


# Calcula o expoente de hurst
def get_hurst_exponent(time_series):
    
    # Definindo o intervalo de taus
    max_tau = round(len(time_series)/4)
    taus = range(2, max_tau)

    # Calculando a variável k
    k = [np.std(np.subtract(time_series[tau:], time_series[:-tau])) for tau in taus]
    
    'To calculate the Hurst exponent, we first calculate the standard deviation of the differences between a series and its lagged version, for a range of possible lags.'

    # Calculate the slope of the log plot -> the Hurst Exponent
    reg = np.polyfit(np.log(taus), np.log(k), 1)
    
    'We then estimate the Hurst exponent as the slope of the log-log plot of the number of lags versus the mentioned standard deviations.'

    return reg[0]

In [ ]:
prices = pd.read_csv("../cointegration_data/Pt_cointegration.csv")
returns = pd.read_csv("../cointegration_data/Rt_cointegration.csv")
periods = pd.read_csv("../distance_data/Periods.csv", header=None)
ticker2 = pd.read_csv("../distance_data/ticker2.csv", header=None)
ticker_b = pd.read_csv("../distance_data/ticker_b.csv", header=None)

print(prices)
print(returns)

In [ ]:
# Conversão dos dados para log dos preços
log_data = np.log(prices)
print(log_data.head())

In [ ]:
no_pairs = 20
days, num_assets = np.shape(returns)
first_traininig = int(periods.iloc[0, 3])
Rpair = np.zeros((days, no_pairs))

In [ ]:
results = []
resumos = []
alpha = 0.05
threshold = 1.5

read_csv = False

past_days=0

for big_loop in range(0, int(years * 2 - 2)):
    print(f"Starting period {big_loop} | Past days: {past_days}")
    
    twelve_months = int(periods.iloc[big_loop, 3])
    six_months = int(periods.iloc[big_loop + 2, 0])
    
    # Limpeza das ações não listadas no período
    listed1 = log_data.iloc[past_days, :] > 0
    listed2 = log_data.iloc[past_days+int(twelve_months+six_months*(s1218 == 1))-1, :] > 0
    listed = np.multiply(listed1, listed2)
    listed_num = np.sum(listed)
    #print(f"Listed stocks for the period: {listed_num}")
    listed_indexes = np.where(listed > 0)[0]
    #print(f"Listed indexes len {np.shape(listed_indexes)} value {listed_indexes}")
    listed_stocks = log_data.columns[listed_indexes]
    
    #print(f"First listed stockes len {np.shape(listed_stocks)} value {listed_stocks}")
    
    [D, ia, ib] = np.intersect1d(
        ticker2.iloc[:, big_loop], ticker2.iloc[:, big_loop+1], return_indices=True)
    
    ic = np.isin(D, ticker2.iloc[:, big_loop+2])
    
    #print(f"ic len: {np.shape(ic)} | variable: {ic}")
    #print(f"D(ic) len: {np.shape(D[ic])} | variable: {D[ic]}")
    
    Dic_unique_sorted, B_idx = np.unique(D[ic], return_index=True)
    #print(f"Dic_unique_sorted len: {np.shape(Dic_unique_sorted)} | variable: {Dic_unique_sorted}")
    
    listed_union = np.intersect1d(listed_stocks,Dic_unique_sorted)
    #print(f"Listed union len {np.shape(listed_union)} value {listed_union}")
    
    index_listed2 = [log_data.columns.get_loc(i) for i in listed_union if i in log_data]
    index_listed2.sort()
    #print(f"Listed union indexes len {np.shape(index_listed2)} value {index_listed2}")
    
    if os.path.exists(f"../cointegration_data/pvalues_semester_{big_loop}.csv"):
        print("Reading csv")
        try:
            pvalues = np.genfromtxt(f"../cointegration_data/pvalues_semester_{big_loop}.csv", delimiter=',')
            #pvalues = pd.read_csv(f"cointegration_data/pvalues_semester_{j}.csv", header=None, index_col=False)
            gammas = np.genfromtxt(f"../cointegration_data/gammas_semester_{big_loop}.csv", delimiter=',')
            #gammas = pd.read_csv(f"cointegration_data/gammas_semester_{j}.csv", header=None, index_col=False)
        except Exception as e:
            print(e)
    else:
        print("Generating csv")
        pvalues, gammas = find_cointegrated_pairs_mod(log_data.iloc[past_days:(past_days+twelve_months),index_listed2]) # 1 ano de formação e 6 meses de teste
        
    #pd.DataFrame(pvalues).to_csv(f"../cointegration_data/pvalues_semester_{big_loop}.csv", header=None, index=False)
    #pd.DataFrame(gammas).to_csv(f"../cointegration_data/gammas_semester_{big_loop}.csv", header=None, index=False)
    
    print(f"Finished finding cointegrated pairs big_loop {big_loop}")

    try:
        coint_pairs_df = top_coint_pairs(log_data.iloc[past_days:(past_days+twelve_months),index_listed2], pvalues, gammas, alpha, no_pairs)
        print(f"Found top cointegrated pairs big_loop {big_loop}")
        #print(f"coint_pairs_df len: {np.shape(coint_pairs_df)} | value: {coint_pairs_df}")
    except Exception as e:
        print(e)
        print("Exception ao achar top pares cointegrados")
        continue
        
    for i in range(0,coint_pairs_df.shape[0]):
        #print(f"Pair: {coint_pairs_df.iloc[i,:]}")
        
        S1_name = coint_pairs_df.iloc[i, 0]
        S2_name = coint_pairs_df.iloc[i, 1]
        gamma_1_2 = coint_pairs_df.iloc[i, 3]

        S1 = log_data[S1_name].iloc[(past_days+twelve_months):(past_days+twelve_months+six_months)] # periodo de teste
        S2 = log_data[S2_name].iloc[(past_days+twelve_months):(past_days+twelve_months+six_months)] # periodo de teste

        # Spread
        spread = S1 - gamma_1_2*S2
        #print(f"Spread series: {spread}")
        # Pegando o resultado da estratégia
        ret, resumos = calculate_profit(i, spread, threshold, S1_name, S2_name, resumos, big_loop, gamma_1_2)
        
    past_days = past_days + periods.iloc[big_loop, 0]
    
    
cols = ['Operação', 'Semestre', 'Abertura', 'Fechamento', 'Dias', 'Retorno total', 'Ticker 1', 'Ticker 2', 'Converged']
df = pd.DataFrame(resumos, columns = cols)
df['Index'] = df['Ticker 1'].astype(str) + '-' + df['Ticker 2'].astype(str) + '-' + df['Operação'].astype(str)
df['Retorno total - exp'] = np.exp(df['Retorno total'])
df.to_csv("../hurst_results/operations_test.csv", sep=',', index=False)


In [ ]:
pd.DataFrame(Rpair).to_csv("../hurst_results/Rpair_test.csv", header=None, index=False)
daily_returns = np.sum(Rpair, axis=1)
pd.DataFrame(daily_returns).to_csv("../hurst_results/daily_returns_test.csv", header=None, index=False)